In [110]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame
import pickle
import os
from dtaidistance import dtw, dtw_visualisation as dtwvis

In [111]:
filenames_patients = {
    "cruz_ej1": [
        "cruz_ej1_0",
        "cruz_ej1_3",
        "cruz_ej1_13",
        "cruz_ej1_15",
        "cruz_ej1_35",
        "cruz_ej1_40",
        "cruz_ej1_55",
        "cruz_ej1_57",
        ],
    "cruz_ej2" : [
        "cruz_ej2_1",
        "cruz_ej2_14",
        "cruz_ej2_19",
        "cruz_ej2_23",
        "cruz_ej2_26",
        "cruz_ej2_28",
        "cruz_ej2_29",
        "cruz_ej2_36",
        "cruz_ej2_41",
        "cruz_ej2_45",
        "cruz_ej2_47",
        "cruz_ej2_49",
        "cruz_ej2_53",
        "cruz_ej2_60",
    ],
    "cruz_ej3" : [
        "cruz_ej3_2",
        "cruz_ej3_6",
        "cruz_ej3_10",
        "cruz_ej3_17",
        "cruz_ej3_59",
    ],
    "cruz_ej4" : [
        "cruz_ej4_30",
        "cruz_ej4_31",
        "cruz_ej4_32",
        "cruz_ej4_34",
        "cruz_ej4_38",
        "cruz_ej4_42",
        "cruz_ej4_51",
        "cruz_ej4_54",
    ],
    "pelota1" : [
        "pelota1_4",
        "pelota1_0",
        "pelota1_11",
        "pelota1_17",
        "pelota1_20",
        "pelota1_24",
        "pelota1_30",
        "pelota1_31",
    ],
    "pelota2": [
        "pelota2_12",
        "pelota2_32",
    ]
}    


filenames_correct = {
    "cruz_ej1" : [
        "josemi_cruz1"
    ],
    "cruz_ej2" : [
        "josemi_cruz2"
    ],
    "cruz_ej3" : [
        "josemi_cruz3",
        "joselu_cruz3"
    ],
    "cruz_ej4" : [
        "josemi_cruz4",
    ],
    "pelota1" : [
        "josemi_pelota1",
        "joselu_pelota1",
        "gadea_pelota1"
    ],
    "pelota2" : [
        "josemi_pelota2",
        "joselu_pelota2",
        "gadea_pelota2"
    ]
}

DATA_DIR = "../datos"
CSV_DIR = "CSV"
DATAFRAME_DIR = "DataFrames"
PROFESIOLS_DIR = "Correctos"
NORMALIZED = "normalized"


def get_filenames_patients(exercise: str):
    sufix = ".pos"
    dataframes = []
    for i in filenames_patients[exercise]:
        actual_df =  i + sufix
        dataframes.append(actual_df)
    return dataframes

def get_filenames_profesionals(exercise: str):
    dataframes = []
    for i in filenames_correct[exercise]:
        actual_df = i
        dataframes.append(actual_df)
    return dataframes

In [112]:
import PosicionVF
from PosicionVF import Posicion


def open_dataframe(df_name):
    df: DataFrame
    df_name = os.path.join(DATA_DIR, DATAFRAME_DIR, df_name) + ".df"
    
    with open(df_name, 'rb') as fp:
        df = pd.read_pickle(fp)
    df = df.dropna(subset=df.columns.values)
    df = df.reset_index(drop=True)
    df = df.drop('frame',axis= 1)
    return df

def open_numpy(numpy_name):
    file = None
    numpy_name = os.path.join(DATA_DIR, PROFESIOLS_DIR, numpy_name) + ".df"
    with open(numpy_name, 'rb') as fp:
        file = pd.read_pickle(fp)
    file = file.dropna(subset=file.columns.values)
    file = file.reset_index(drop=True)
    file = file.drop('frame',axis= 1)
    return file

In [113]:

def save_file_profesional(data, filename):
    filename_df =  os.path.join(NORMALIZED, PROFESIOLS_DIR, filename) + ".df"   
    with open(filename_df, "wb") as file:
        pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)
    
    filename_csv =  os.path.join(NORMALIZED, PROFESIOLS_DIR, filename) + ".csv"
    data.to_csv(filename_csv)

def save_file_patient(data, filename):
    filename_df =  os.path.join(NORMALIZED, DATAFRAME_DIR, filename) + ".df"
    with open(filename_df, "wb") as file:
        pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

    filename_csv =  os.path.join(NORMALIZED, CSV_DIR, filename) + ".csv"
    data.to_csv(filename_csv)

In [114]:
ang = ['angCuelloI',
       'angCuelloD', 'angCodoI', 'angCodoD', 'angHombroI', 'angHombroD',
       'angCaderaI', 'angCaderaD', 'angCaderaTorsoI', 'angCaderaTorsoD',
       'angRodillaI', 'angRodillaD']

In [115]:
def mini_max_angles(data):    
    for i in ang:
        max = np.max(data[i])
        min = np.min(data[i])

        data[i] =(data[i] - min) / (max - min)

def zscore_angles(data):
    from scipy.stats import zscore
    for i in ang:
        data[i] = zscore(data[i])


def normalize_angles(data, mode = "minmax"):
    data[ang] = data[ang].astype(float)
    if mode == "minmax":
        return mini_max_angles(data)
    elif mode == "zscore":
        zscore_angles(data)
    else:
        for i in ang:
            data[i] = data[i]/360



In [116]:
body = ['nariz', 'cuello', 'hombroI', 'hombroD',
       'codoI', 'codoD', 'manoI', 'manoD', 'caderaI', 'caderaD',
       'caderaM', 'rodillaI', 'rodillaD', 'tobilloI', 'tobilloD']

def get_positions():
    positions = []
    for i in body:
        positions.append(i + "_x")
        positions.append(i + "_y")
    return positions

def get_positions_x():
        positions = []
        for i in body:
            positions.append(i + "_x")
        return positions

def get_positions_y():
        positions = []
        for i in body:
            positions.append(i + "_y")
        return positions

In [117]:
def bounding_box(row):

		min_x = 1000000
		min_y = 1000000
		max_x = -1000000
		max_y = -1000000

		for i in body:
			coord_x = row[i + "_x"]
			coord_y = row[i + "_y"]
			if coord_x < min_x:
				min_x = coord_x

			if coord_x > max_x:
				max_x = coord_x

			if coord_y < min_y:
				min_y = coord_y

			if coord_y > max_y:
				max_y = coord_y
		return [(int(min_x),int(min_y)),(int(max_x),int(min_y)),(int(max_x),int(max_y)),(int(min_x),int(max_y))]

def normalize_positions(data):
	data[get_positions()] = data[get_positions()].astype(float)
	selected_columns = data[get_positions()]
	for index, row in selected_columns.iterrows():
		coords = bounding_box(row)
		for i in body:
			x = i + "_x"
			y = i + "_y"
			data.loc[index, x] = np.float64(row[x] - coords[0][0])
			data.loc[index, y] = np.float64(row[y] - coords[0][1])
	for i in body:
		columns = data[[i + "_x", i + "_y"]]
		data[[i + "_x", i + "_y"]] = columns/np.linalg.norm(columns)
		#print(columns)


In [118]:
for i in filenames_patients.keys():
    filenames = get_filenames_patients(i) 
    for filename in filenames:
        temp = open_dataframe(filename)
        normalize_angles(temp)
        normalize_positions(temp)
        save_file_patient(temp, filename)
        print(filename + " normalizado")

cruz_ej1_0.pos normalizado
cruz_ej1_3.pos normalizado
cruz_ej1_13.pos normalizado
cruz_ej1_15.pos normalizado
cruz_ej1_35.pos normalizado
cruz_ej1_40.pos normalizado
cruz_ej1_55.pos normalizado
cruz_ej1_57.pos normalizado
cruz_ej2_1.pos normalizado
cruz_ej2_14.pos normalizado
cruz_ej2_19.pos normalizado
cruz_ej2_23.pos normalizado
cruz_ej2_26.pos normalizado
cruz_ej2_28.pos normalizado
cruz_ej2_29.pos normalizado
cruz_ej2_36.pos normalizado
cruz_ej2_41.pos normalizado
cruz_ej2_45.pos normalizado
cruz_ej2_47.pos normalizado
cruz_ej2_49.pos normalizado
cruz_ej2_53.pos normalizado
cruz_ej2_60.pos normalizado
cruz_ej3_2.pos normalizado
cruz_ej3_6.pos normalizado
cruz_ej3_10.pos normalizado
cruz_ej3_17.pos normalizado
cruz_ej3_59.pos normalizado
cruz_ej4_30.pos normalizado
cruz_ej4_31.pos normalizado
cruz_ej4_32.pos normalizado
cruz_ej4_34.pos normalizado
cruz_ej4_38.pos normalizado
cruz_ej4_42.pos normalizado
cruz_ej4_51.pos normalizado
cruz_ej4_54.pos normalizado
pelota1_4.pos normalizado

In [119]:
for i in filenames_correct.keys():
    filenames = get_filenames_profesionals(i) 
    for filename in filenames:
        temp = open_numpy(filename)
        normalize_angles(temp)
        normalize_positions(temp)
        save_file_profesional(temp, filename)
        print(filename + " normalizado")

josemi_cruz1 normalizado
josemi_cruz2 normalizado
josemi_cruz3 normalizado
joselu_cruz3 normalizado
josemi_cruz4 normalizado
josemi_pelota1 normalizado
joselu_pelota1 normalizado
gadea_pelota1 normalizado
josemi_pelota2 normalizado
joselu_pelota2 normalizado
gadea_pelota2 normalizado
